# Save Player Cumulative Season BoxScores 

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import time
from nba_api.stats.endpoints import leaguedashplayerstats, leaguegamelog

box_DIR = "../../data/box/"

In [ ]:
season_start = 1991
season_end = 1992
seasons = np.arange(season_start,season_end,1).astype(str)

league = "NBA"

# Cumulative Boxscores

In [ ]:
def get_box_p_cum(seasons, measure="Base", n=32):
    if measure == "Advanced":
        per_mode = "Per100Possessions"
    else:
        per_mode = "PerGame"
    for season in tqdm(seasons):
        # try:
            season_str = season + "-" + str(int(season) + 1)[-2:]
            stats = leaguedashplayerstats.LeagueDashPlayerStats(
                measure_type_detailed_defense=measure,
                per_mode_detailed=per_mode,
                season=season_str,
            )
            df1 = stats.get_data_frames()[0]
            df = df1.iloc[:, :n]
            if measure == "Advanced":
                measure1 = "Adv"
            else:
                 measure1 = measure
            df.to_parquet(box_DIR + "NBA_Box_P_Cum_" + measure1 + "_" + season + ".parquet")
            time.sleep(0.6)
        # except Exception as error:
        #     continue

In [ ]:
def update_box_p_cum(seasons):
    print("Base")
    get_box_p_cum(seasons, measure="Base", n=32)
    print("Advanced")
    get_box_p_cum(seasons, measure="Advanced", n=43)
    print("Misc")
    get_box_p_cum(seasons, measure="Misc", n=23)
    print("Scoring")
    get_box_p_cum(seasons, measure="Scoring", n=29)

In [ ]:
update_box_p_cum(seasons)

# Individual Game BoxScores

In [ ]:
def update_box_base_p(seasons):
    for season in tqdm(seasons):
        try:
            stats = leaguegamelog.LeagueGameLog(
                player_or_team_abbreviation="P",
                season=season,
                season_type_all_star="Regular Season",
            )
            df = stats.get_data_frames()[0]
            df.to_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + season + ".parquet")
            time.sleep(0.6)
        except Exception as error:
            continue

In [ ]:
update_box_base_p(seasons)

In [ ]:
dfa = []
for year in range(1946,2024):
    df1 = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1[df1["WL"] == 0]["WL"] = "L"
    df1["season"] = year+1
    df1 = df1.fillna(0)
    dfa.append(df1)
df = pd.concat(dfa)
df.loc[df[df["WL"] == 0].index,"WL"] = "L"
df.to_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

In [ ]:
df = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df.columns = map(str.lower, df.columns)